In [19]:
import sys
sys.path.append("..")

In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
from src.dataset import HierarchicalDataset
import pandas as pd

In [24]:
dataset = HierarchicalDataset(cases_dir="./ILCaseAndMortalityInputV1.csv",
                              ifr_dir="../../data/EU/weighted_fatality.csv", 
                              serial_interval_dir="../../data/EU/serial_interval.csv", 
                              interventions_dir="../../data/EU/interventions.csv"
                             )

covariate names:
schools_universities
travel_restrictions
public_events
sport
social_distancing_encouraged
covariates:
           Country schools_universities travel_restrictions public_events  \
0          Austria           2020-03-14          2020-03-13    2020-03-10   
1          Belgium           2020-03-14          2020-03-18    2020-03-12   
2          Denmark           2020-03-13          2020-03-13    2020-03-12   
3           France           2020-03-14          2020-03-17    2020-03-13   
4          Germany           2020-03-14          2020-03-16    2020-03-22   
5            Italy           2020-03-05          2020-03-09    2020-03-09   
6           Norway           2020-03-13          2020-03-13    2020-03-12   
7            Spain           2020-03-13          2020-03-12    2020-03-14   
8           Sweden           2020-03-18          2020-03-19    2020-03-12   
9      Switzerland           2020-03-14          2020-03-18    2020-03-13   
10  United_Kingdom           2020-

../src/dataset.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
stan_data = dataset.get_stan_data(75)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [6]:
stan_data

{'M': 11,
 'p': 7,
 'x1': array([-1.97350876e-01, -1.92017069e-01, -1.86683261e-01, -1.81349454e-01,
        -1.76015647e-01, -1.70681839e-01, -1.65348032e-01, -1.60014224e-01,
        -1.54680417e-01, -1.49346609e-01, -1.44012802e-01, -1.38678994e-01,
        -1.33345187e-01, -1.28011379e-01, -1.22677572e-01, -1.17343764e-01,
        -1.12009957e-01, -1.06676149e-01, -1.01342342e-01, -9.60085345e-02,
        -9.06747270e-02, -8.53409195e-02, -8.00071121e-02, -7.46733046e-02,
        -6.93394971e-02, -6.40056896e-02, -5.86718822e-02, -5.33380747e-02,
        -4.80042672e-02, -4.26704598e-02, -3.73366523e-02, -3.20028448e-02,
        -2.66690374e-02, -2.13352299e-02, -1.60014224e-02, -1.06676149e-02,
        -5.33380747e-03,  2.64323941e-18,  5.33380747e-03,  1.06676149e-02,
         1.60014224e-02,  2.13352299e-02,  2.66690374e-02,  3.20028448e-02,
         3.73366523e-02,  4.26704598e-02,  4.80042672e-02,  5.33380747e-02,
         5.86718822e-02,  6.40056896e-02,  6.93394971e-02,  7.4

In [7]:
# remove travel bans and sport
stan_data["covariate2"] = 0*stan_data["covariate2"]
stan_data["covariate4"] = 0*stan_data["covariate4"]

# self-isolating if ill
stan_data["covariate2"] = stan_data["covariate7"]

In [8]:
#create the any intervention covariate
stan_data["covariate4"] = 1*(stan_data["covariate1"]+
                            stan_data["covariate3"]+
                            stan_data["covariate4"]+
                            stan_data["covariate5"]+
                            stan_data["covariate6"] > 1)

In [9]:
stan_data["covariate7"]=0*stan_data["covariate7"]

In [10]:
import pystan


In [11]:
sm = pystan.StanModel(file="../../stan-models/base.stan")

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_cac77d4c122b5d474c43fdbfc9c71646 NOW.


In [14]:
# original
# fit = sm.sampling(data=stan_data,iter=40,warmup=20,chains=2)

# sample as small as possible, just for testing
fit = sm.sampling(data=stan_data,iter=2,warmup=1,chains=1)
print(fit)

To run all diagnostics call pystan.check_hmc_diagnostics(fit)
/Users/mattgarvin/.local/share/virtualenvs/Python-oEzEVjUf/lib/python3.8/site-packages/pystan/diagnostics.py:217: RuntimeWarning: invalid value encountered in true_divide
  e_bfmi = numer / denom
/Users/mattgarvin/.local/share/virtualenvs/Python-oEzEVjUf/lib/python3.8/site-packages/numpy/core/_methods.py:216: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/mattgarvin/.local/share/virtualenvs/Python-oEzEVjUf/lib/python3.8/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Inference for Stan model: anon_model_cac77d4c122b5d474c43fdbfc9c71646.
1 chains, each with iter=2; warmup=1; thin=1; 
post-warmup draws per chain=1, total post-warmup draws=1.

                       mean se_mean     sd     2.5%      25%      50%      75%    97.5%  n_eff   Rhat
mu[1]                  1.02     nan    nan     1.02     1.02     1.02     1.02     1.02    nan    nan
mu[2]                  0.94     nan    nan     0.94     0.94     0.94     0.94     0.94    nan    nan
mu[3]                  1.35     nan    nan     1.35     1.35     1.35     1.35     1.35    nan    nan
mu[4]                  6.69     nan    nan     6.69     6.69     6.69     6.69     6.69    nan    nan
mu[5]                  0.51     nan    nan     0.51     0.51     0.51     0.51     0.51    nan    nan
mu[6]                  4.53     nan    nan     4.53     4.53     4.53     4.53     4.53    nan    nan
mu[7]                  0.75     nan    nan     0.75     0.75     0.75     0.75     0.75    nan    nan
mu[8]  

In [15]:
# save it to the file for later use
import pickle
with open('./example.pkl', 'wb') as f:
    pickle.dump(sm, f)

In [16]:
# import arviz as az
fit.extract()

OrderedDict([('mu',
              array([[1.0152931 , 0.93656017, 1.3469717 , 6.69454756, 0.5125253 ,
                      4.52857337, 0.74809275, 2.60462237, 0.50817787, 0.33251021,
                      2.64582928]])),
             ('alpha',
              array([[0.99916185, 0.41968745, 2.26352843, 2.82178925, 4.59475762,
                      0.22998168]])),
             ('kappa', array([0.15980838])),
             ('y',
              array([[3.81825072, 6.49353648, 5.80771001, 0.28340415, 6.95876193,
                      0.95241796, 0.60016409, 0.29102028, 1.47579156, 1.83390853,
                      1.41589948]])),
             ('phi', array([1.68923211])),
             ('tau', array([2.99087265])),
             ('convolution', array([2.38771408e-05])),
             ('prediction',
              array([[[3.81825072e+00, 6.49353648e+00, 5.80771001e+00, 2.83404145e-01,
                       6.95876193e+00, 9.52417958e-01, 6.00164095e-01, 2.91020276e-01,
                       1.4